In [1]:
import logging
import torch
import transformers
from transformers import T5Tokenizer, T5EncoderModel, T5Config

/home/ubuntu/anaconda3/envs/muse/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/ubuntu/anaconda3/envs/muse/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
MAX_LENGTH = 256
DEFAULT_T5_NAME = 'google/t5-v1_1-base'
T5_CONFIGS = {}

In [3]:
def get_tokenizer(name):
  tokenizer = T5Tokenizer.from_pretrained(name)
  return tokenizer

In [4]:
def get_model(name):
  model = T5EncoderModel.from_pretrained(name)
  return model

In [5]:
def get_model_and_tokenizer(name):
  global T5_CONFIGS

  if name not in T5_CONFIGS:
    T5_CONFIGS[name] = dict()
  if "model" not in T5_CONFIGS[name]:
    T5_CONFIGS[name]["model"] = get_model(name)
  if "tokenizer" not in T5_CONFIGS[name]:
    T5_CONFIGS[name]["tokenizer"] = get_tokenizer(name)
  
  return T5_CONFIGS[name]["model"], T5_CONFIGS[name]["tokenizer"]

In [6]:
def get_encoded_dim(name):
  if name not in T5_CONFIGS:
    config = T5Config.from_pretrained(name)
    T5_CONFIGS["name"] = dict(config = config)
  elif "config" in T5_CONFIGS[name]:
    config = T5_CONFIGS[name]["config"]
  elif "model" in T5_CONFIGS[name]:
    config = T5_CONFIGS[name]["model"].config
  else:
    assert False
  return config.d_model

In [7]:
get_encoded_dim(DEFAULT_T5_NAME)

768

In [8]:
texts = ["today tomorrow friday"]

In [9]:
def t5_encode_text(
    texts,
    name = DEFAULT_T5_NAME,
    output_device = None,
):
  if isinstance(texts, str):
    texts = [texts]

  t5, tokenizer = get_model_and_tokenizer(DEFAULT_T5_NAME)

  if torch.cuda.is_available():
    t5 = t5.cuda()

  device = next(t5.parameters()).device

  encoded = tokenizer.batch_encode_plus(
    texts,
    return_tensors = "pt",
    padding = "longest",
    max_length = MAX_LENGTH,
    truncation = True
  )

  input_ids = encoded.input_ids.to(device)
  attn_mask = encoded.attention_mask.to(device)

  t5.eval()

  with torch.no_grad():
    output = t5(input_ids = input_ids, attention_mask = attn_mask)
    encoded_text = output.last_hidden_state.detach()

  attn_mask = attn_mask.bool()
  encoded_text = encoded_text.masked_fill(~attn_mask[..., None], 0.)

  if output_device is None:
    return encoded_text
  
  encoded_text.to(output_device)
  return encoded_text